<a href="https://colab.research.google.com/github/Choi-SeoYun/2020DACONCUP/blob/main/%5BSY%5DDaconCup_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# base line 따라하기

## 1. library import

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)
# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 2. 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train = pd.read_csv("/content/gdrive/MyDrive/open_data/train.csv", encoding = 'euc-kr')
train.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 00:00:00,19,19,8,206
1,2018-09-09 01:00:00,20,19,9,259
2,2018-09-09 02:00:00,12,9,1,48
3,2018-09-09 03:00:00,10,10,2,102
4,2018-09-09 04:00:00,6,5,3,18


In [ ]:
train['DateTime'] = pd.to_datetime(train.DateTime) # 이거 왜 한거지? 아까랑 달라진게 없지 않나?
train.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 00:00:00,19,19,8,206
1,2018-09-09 01:00:00,20,19,9,259
2,2018-09-09 02:00:00,12,9,1,48
3,2018-09-09 03:00:00,10,10,2,102
4,2018-09-09 04:00:00,6,5,3,18


In [ ]:
train['date'] = train.DateTime.dt.date # 여기서 dt가 뭐지?
train.head()

,DateTime,사용자,세션,신규방문자,페이지뷰,date
0,2018-09-09 00:00:00,19,19,8,206,2018-09-09
1,2018-09-09 01:00:00,20,19,9,259,2018-09-09
2,2018-09-09 02:00:00,12,9,1,48,2018-09-09
3,2018-09-09 03:00:00,10,10,2,102,2018-09-09
4,2018-09-09 04:00:00,6,5,3,18,2018-09-09


In [ ]:
train  = train.groupby('date').sum().reset_index()
# date로 묶고 같은 date이면 sum한다.
# reset_index()를 하면 처음부터 index를 해준다. (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html 참고)
# 근데 DateTime은 어디간거지? 
train.head()

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,281,266,73,1826
1,2018-09-10,264,247,51,2092
2,2018-09-11,329,310,58,1998
3,2018-09-12,300,287,45,2595
4,2018-09-13,378,344,50,3845


## 3. 데이터 수정

In [ ]:
# scaling
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()
train.iloc[:,1:] = (train.iloc[:,1:] -  mini) / size

input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values

## 4. 모델 생성

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size = hidden_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)
    
model = LSTM(input_size = 4, hidden_size = 30).to(device)

## 5. 학습

In [ ]:
window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.MSELoss(size_average = True)
num_epochs  = 600
train_error = []
for t in range(num_epochs):
    train_pred = model(window_x)
    loss = criterion(train_pred, window_y) ### trend
    train_error.append(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if t % 10 == 0 and t !=0:
        print(f"{t} Epochs train MSE: {loss.item():1.5f}")

10 Epochs train MSE: 0.00574
20 Epochs train MSE: 0.00569
30 Epochs train MSE: 0.00565
40 Epochs train MSE: 0.00559
50 Epochs train MSE: 0.00553
60 Epochs train MSE: 0.00548
70 Epochs train MSE: 0.00543
80 Epochs train MSE: 0.00538
90 Epochs train MSE: 0.00534
100 Epochs train MSE: 0.00528
110 Epochs train MSE: 0.00520
120 Epochs train MSE: 0.00511
130 Epochs train MSE: 0.00502
140 Epochs train MSE: 0.00501
150 Epochs train MSE: 0.00487
160 Epochs train MSE: 0.00480
170 Epochs train MSE: 0.00471
180 Epochs train MSE: 0.00463
190 Epochs train MSE: 0.00460
200 Epochs train MSE: 0.00452
210 Epochs train MSE: 0.00448
220 Epochs train MSE: 0.00445
230 Epochs train MSE: 0.00448
240 Epochs train MSE: 0.00438
250 Epochs train MSE: 0.00435
260 Epochs train MSE: 0.00431
270 Epochs train MSE: 0.00434
280 Epochs train MSE: 0.00428
290 Epochs train MSE: 0.00423
300 Epochs train MSE: 0.00420
310 Epochs train MSE: 0.00423
320 Epochs train MSE: 0.00414
330 Epochs train MSE: 0.00412
340 Epochs train MS

## 6. 예측

In [ ]:
submission = pd.read_csv("/content/gdrive/MyDrive/open_data/submission.csv", encoding = 'euc-kr')

#last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device) <- 수정 전
last_month = train.iloc[-30:,1:].values[np.newaxis,...] # <- 수정 후
last_month = torch.tensor(last_month).float().to(device) # <- 수정 후

for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    next_week = model(last_month)
    #last_month = torch.cat([last_month[-7:], next_week], axis = 1) <- 수정 전
    last_month = torch.cat([last_month[:,7:,:], next_week], axis = 1)# <- 수정 후

    pred_week = next_week.cpu().detach().numpy().reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,3133,3128,737,86306
1,2020-11-10,3395,3343,797,89317
2,2020-11-11,2833,2718,676,70421
3,2020-11-12,2459,2382,599,58393
4,2020-11-13,2095,2016,519,47713
...,...,...,...,...,...
56,2021-01-04,1430,1314,245,24734
57,2021-01-05,1365,1287,281,26489
58,2021-01-06,1391,1246,262,31513
59,2021-01-07,1377,1141,214,31821


## 7. 예측파일 저장

In [ ]:
submission.to_csv('submission.csv', index = False, encoding = 'euc-kr')